## Building and Serving an OpenAI-compatible Chatbot

Welcome to our tutorial on using Transformers and MLflow to create an OpenAI-compatible chat model. In Mlflow 2.11 and up, the [ChatModel](https://mlflow.org/docs/latest/python_api/mlflow.types.html#) class has been added, allowing for convenient creation of served models that conform to the OpenAI API spec. This enables you to seamlessly swap out your chat app's backing LLM, or to easily evaluate different models without having to edit your client-side code.

If you haven't already seen it, you may find it helpful to go through our [introductory notebook on chat and Transformers](https://mlflow.org/docs/latest/llms/transformers/tutorials/conversational/conversational-model.html) before proceeding with this one, as this notebook is slightly higher-level and does not delve too deeply into the inner workings of Transformers or MLflow Tracking.

### Learning objectives

In this tutorial, you will:

- Create an OpenAI-compatible chat model using TinyLLama-1.1B-Chat
- Serve the model with MLflow Model Serving
- Learn how to use MLflow's `pyfunc` API to add arbitrary customization to your model

In [1]:
# Disable tokenizers warnings when constructing pipelines
%env TOKENIZERS_PARALLELISM=false

import warnings

# Disable a few less-than-useful UserWarnings from setuptools and pydantic
warnings.filterwarnings("ignore", category=UserWarning)

env: TOKENIZERS_PARALLELISM=false


### Serving Chat Model with Transformers

MLflow's native Transformers integration allows you to specify the `task` param when saving or logging your `text-generation` pipelines. This param can be one of two string literals, conforming to the [MLflow Deployments Server's endpoint_type specification](https://mlflow.org/docs/latest/llms/deployments/index.html#general-configuration-parameters) ("llm/v1/embeddings" can be specified as a task on models saved with `mlflow.sentence_transformers`):

- "llm/v1/chat" for chat-style applications
- "llm/v1/completions" for generic completions

When the `task` param is specified, MLflow will automatically handle everything required to serve a chat model. This includes:

- Setting a chat-compatible signature on the model
- Performing data pre- and post-processing to ensure the inputs and outputs conform to the [Chat API spec](https://mlflow.org/docs/latest/llms/deployments/index.html#chat), which is compatible with OpenAI's API spec.

Note that these modifications only apply when the model is loaded with `mlflow.pyfunc.load_model()` (e.g. when serving the model with the `mlflow models serve` CLI tool). If you want to load just the base pipeline, you can always do so via `mlflow.transformers.load_model()`.

Here's an example of how to save a Transformers model with the `task` param using TinyLlama-1.1B-Chat:

In [3]:
from transformers import pipeline

import mlflow

generator = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
)

mlflow.transformers.save_model(
    transformers_model=generator, path="tinyllama-chat", task="llm/v1/chat"
)

/var/folders/qd/9rwd0_gd0qs65g4sdqlm51hr0000gp/T/ipykernel_74928/2826724314.py:10: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.37.1``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  mlflow.transformers.save_model(


Once the model is saved, we simply have to serve the model using the `mlflow models serve` CLI tool. 

In a terminal shell, run:
```
$ mlflow models serve -m tinyllama-chat
```

When the server has finished initializing, you should be able to interact with the model via HTTP requests. The input format is almost identical to the format described in the [Mlflow Deployments Server docs](https://mlflow.org/docs/latest/llms/deployments/index.html#chat), with the exception that `temperature` defaults to `1.0` instead of `0.0`.

Here's a quick example:

In [5]:
%%sh
curl http://127.0.0.1:5000/invocations \
    -H 'Content-Type: application/json' \
    -d '{ "messages": [{"role": "user", "content": "Write me a hello world program in python"}] }' \
    | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   706  100   617  100    89     30      4  0:00:22  0:00:20  0:00:02   191


[
  {
    "id": "45f7b834-07d8-42bf-bae0-35d21d0578f8",
    "object": "chat.completion",
    "created": 1708918105,
    "model": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    "usage": {
      "prompt_tokens": 24,
      "completion_tokens": 71,
      "total_tokens": 95
    },
    "choices": [
      {
        "index": 0,
        "finish_reason": "stop",
        "message": {
          "role": "assistant",
          "content": "Here's a simple hello world program in Python:\n\n```python\nprint(\"Hello, world!\")\n```\n\nThis program prints the string \"Hello, world!\" to the console. You can run this program by typing it into the Python interpreter or by running the command `python hello_world.py` in your terminal."
        }
      }
    ]
  }
]


It's that easy!

You can also call the API with a few optional inference params to adjust the model's responses. These map to Transformers pipeline params, and are passed in directly at inference time.

- `max_tokens` (maps to `max_new_tokens`): The maximum number of new tokens the model should generate.
- `temperature` (maps to `temperature`): Controls the creativity of the model's response. Note that this is not guaranteed to be supported by all models, and in order for this param to have an effect, the pipeline must have been created with `do_sample=True`.
- `stop` (maps to `stopping_criteria`): A list of tokens at which to stop generation.

Note: `n` does not have an equivalent Transformers pipeline param, and is not supported in queries. However, you can implement a model that consumes the `n` param using Custom Pyfunc (details below).

### Customizing the model

As always, custom functionality can be achieved with Mlflow's `pyfunc` API. In the cell below, we create a custom Chat-flavored `pyfunc` model by subclassing `mlflow.pyfunc.ChatModel`. We use our previously-saved TinyLlama pipeline as the backing model by loading it with `mlflow.transformers.load_model()`, and then build our own customizations on top of it.

The possibilities are endless here, but as a quick example, the code below simply edits the ID of the response, rather than having it be a random UUID. Of course, you could also insert any side-effects you wanted here, such as asynchronously logging some metadata for analytics.

In [6]:
import random

import mlflow
from mlflow.types.llm import ChatResponse


class MyChatModel(mlflow.pyfunc.ChatModel):
    def load_context(self, context):
        # load our previously-saved Transformers pipeline from context.artifacts
        self.pipeline = mlflow.transformers.load_model(context.artifacts["chat_model_path"])

    def predict(self, context, messages, params):
        tokenizer = self.pipeline.tokenizer
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        # perform inference using the loaded pipeline
        output = self.pipeline(prompt, return_full_text=False, generation_kwargs=params.to_dict())
        text = output[0]["generated_text"]
        id = f"some_meaningful_id_{random.randint(0, 100)}"

        # construct token usage information
        prompt_tokens = len(tokenizer.encode(prompt))
        completion_tokens = len(tokenizer.encode(text))
        usage = {
            "prompt_tokens": prompt_tokens,
            "completion_tokens": completion_tokens,
            "total_tokens": prompt_tokens + completion_tokens,
        }

        # here, we can do any post-processing or side-effects required.
        # for example, we could log the generated text to a database for
        # analytics, or check the output for any banned words or phrases
        # and return a different response if any are found.

        # in this example, we just return the generated text as the response

        response = {
            "id": id,
            "model": "MyChatModel",
            "choices": [
                {
                    "index": 0,
                    "message": {"role": "assistant", "content": text},
                    "finish_reason": "stop",
                }
            ],
            "usage": usage,
        }

        return ChatResponse(**response)

Similar to what happens above, upon saving an instance of `MyChatModel`, MLflow will automatically recognize the `mlflow.pyfunc.ChatModel` subclass, and set chat signatures and handle input and output parsing automatically. Note that enforcement is performed on the output--MLflow will run inference on an example input, and assert that the output is of type `ChatResponse`.

Full documentation for the `ChatResponse` type can be found in the [API reference](https://mlflow.org/docs/latest/python_api/mlflow.types.html#mlflow.types.llm.ChatResponse).

In [7]:
mlflow.pyfunc.save_model(
    path="my_model",
    python_model=MyChatModel(),
    # provide the path to the pipeline we saved earlier
    artifacts={"chat_model_path": "tinyllama-chat"},
)

2024/02/26 12:30:53 INFO mlflow.pyfunc: Predicting on input example to validate output
/var/folders/qd/9rwd0_gd0qs65g4sdqlm51hr0000gp/T/ipykernel_74928/2688517701.py:9: FutureWarning: The 'transformers' MLflow Models integration is known to be compatible with the following package version ranges: ``4.25.1`` -  ``4.37.1``. MLflow Models integrations with transformers may not succeed when used with package versions outside of this range.
  self.pipeline = mlflow.transformers.load_model(context.artifacts["chat_model_path"])


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

2024/02/26 12:30:53 WARNING mlflow.transformers: Could not specify device parameter for this pipeline type


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

2024/02/26 12:31:23 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


As before, we can now serve the model by running the following in a terminal shell:

```
$ mlflow models serve -m my_model
```

And we should now be able to query it via HTTP request:

In [8]:
%%sh
curl http://127.0.0.1:5000/invocations \
    -H 'Content-Type: application/json' \
    -d '{ "messages": [{"role": "user", "content": "Write me a hello world program in python"}] }' \
    | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   666  100   577  100    89     20      3  0:00:29  0:00:28  0:00:01   171


{
  "id": "some_meaningful_id_21",
  "model": "MyChatModel",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Here's a simple hello world program in Python:\n\n```python\nprint(\"Hello, world!\")\n```\n\nThis program prints the string \"Hello, world!\" to the console. You can run this program by typing it into the Python interpreter or by running the command `python hello_world.py` in your terminal."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 25,
    "completion_tokens": 71,
    "total_tokens": 96
  },
  "object": "chat.completion",
  "created": 1708918478
}


## Conclusion

In this tutorial, you learned how to create an OpenAI-compatible chat model by specifying the `task` parameter when saving Transformers pipelines. You also learned how to leverage Custom Pyfunc to add customizations that fit your specific use-case.

### What's next?

- [In-depth Pyfunc Walkthrough](https://mlflow.org/docs/latest/traditional-ml/creating-custom-pyfunc/index.html). We briefly touched on custom `pyfunc` in this tutorial, but if you're looking for more detail on the anatomy of a `pyfunc` model, the linked page provides an in-depth overview of all the components.
- [More on MLflow Deployments](https://mlflow.org/docs/latest/deployment/index.html). In this tutorial, we saw how to deploy a model using a local server, but MLflow provides many other ways to deploy your models to production. Check out this page to learn more about the different options.
- [More on MLflow's Transformers Integration](https://mlflow.org/docs/latest/llms/transformers/index.html). This page provides a comprehensive overview on MLflow's Transformers integrations, along with lots of hands-on guides and notebooks. Learn how to fine-tune models, use prompt templates, and more!
- [Other LLM Integrations](https://mlflow.org/docs/latest/llms/index.html). Aside from Transformers, MLflow has integrations with many other popular LLM libraries, such as Langchain and OpenAI.